# Palliative Care Cost Model

In [1]:
total_population_input = '326504'

In [2]:
eligible_population_input = '6360'

In [3]:
target_enrollment_percent_input = '20'

In [4]:
initial_enrollment_percent_input = '10'

In [5]:
monthly_growth_rate_input = '12.5'

In [6]:
monthly_attrition_rate_input = '5'

In [7]:
hospice_conversion_rate_input = '37'

In [8]:
projection_months_input = '60'

### Calculate projections & display the output tables

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm
cmap = matplotlib.cm.get_cmap('Reds')
from IPython.display import display, HTML
import numpy as np

# Set Seaborn style for clean visuals
sns.set(style="whitegrid")


# Enter inputs
total_population = int(total_population_input)
eligible_population = int(eligible_population_input)
target_enrollment_percent = float(target_enrollment_percent_input) / 100
initial_enrollment_percent = float(initial_enrollment_percent_input) / 100
monthly_growth_rate = float(monthly_growth_rate_input) / 100
monthly_attrition_rate = float(monthly_attrition_rate_input) / 100
hospice_conversion_rate = float(hospice_conversion_rate_input) / 100
projection_months = int(projection_months_input)

# Outputs
output_target_enrollment = round(eligible_population * target_enrollment_percent)
initial_enrollment = round(output_target_enrollment * initial_enrollment_percent)

display(HTML(f"<h3><b>(Target) Percent of eligibles eventually enrolled (OUTPUT):<b/> {output_target_enrollment}</h3>"))
display(HTML(f"<h3><b>Estimated % of target that will be enrolled in first month (OUTPUT):<b/> {initial_enrollment}</h3>"))

In [10]:
# Function to calculate projections
def calculate_projections_fixed(eligible_population, target_enrollment_percent, initial_enrollment_percent, 
                                 monthly_growth_rate, monthly_attrition_rate, hospice_conversion_rate, projection_months):
    table = []

    # Month 1 - Initial Enrollment
    initial_enrollment = round(eligible_population * target_enrollment_percent * initial_enrollment_percent)
    table.append({
        "New Enrollments": initial_enrollment,
        "Departures": 0,
        "Census": initial_enrollment,
        "Hospice Conversions": 0
    })

    # Loop through the rest of the months
    for month in range(1, projection_months):
        prev = table[month - 1]

        # New Enrollments
        new_enrollment = round(monthly_growth_rate * prev["Census"])

        # Departures
        departure = round(monthly_attrition_rate * prev["Census"])

        # Hospice Conversions (Corrected)
        hospice_conversion = round(departure * hospice_conversion_rate)

        # Census (Corrected to ensure proper cumulative values)
        current_census = max(0, min(eligible_population, prev["Census"] + new_enrollment - departure))

        # Append to table
        table.append({
            "New Enrollments": new_enrollment,
            "Departures": departure,
            "Census": current_census,
            "Hospice Conversions": hospice_conversion
        })

    # Create DataFrame
    df = pd.DataFrame(table)
    df['Year'] = [(i // 12) + 1 for i in range(projection_months)]
    df['Month'] = [(i % 12) + 1 for i in range(projection_months)]
    return df

# Function to calculate annual and multi-year totals (Updated for accurate Annual Total calculation)
def calculate_annual_and_multi_year_totals(projection_df):
    metrics = ["New Enrollments", "Departures", "Census", "Hospice Conversions"]
    projection_df['Year'] = projection_df['Year'].astype(int)
    
    for metric in metrics:
        # Annual Total (Corrected: Explicit sum of monthly values)
        projection_df[f'Annual Total {metric}'] = (
            projection_df.groupby('Year')[metric].transform('sum')
        )

        # Multi-Year Total
        projection_df[f'Multi-Year Total {metric}'] = (
            projection_df.groupby('Year')[f'Annual Total {metric}']
            .transform(lambda x: x.cumsum())
        )

        # Ensure Multi-Year Total is null for the first year
        projection_df.loc[projection_df['Year'] == 1, f'Multi-Year Total {metric}'] = None

        # For Census, set Annual and Multi-Year Total to "NA" or "0" for all months
        if metric == "Census":
            projection_df[f'Annual Total {metric}'] = "NA"
            projection_df[f'Multi-Year Total {metric}'] = "NA"

    return projection_df

# Format numbers
def format_numbers(value):
    if isinstance(value, str) and value == "NA":
        return "NA"
    if value >= 1000:
        return "{:,}".format(int(value))
    return str(int(value))

# Visualize projections
def visualize_projections_fixed(df):
    html_output = ""
    for year in df['Year'].unique():
        yearly_data = df[df['Year'] == year]
        metrics = ['New Enrollments', 'Departures', 'Census', 'Hospice Conversions']
        table = pd.DataFrame(index=metrics, columns=[f'Month {i}' for i in range(1, 13)] + ['Annual Total', 'Multi-Year Total'])

        for metric in metrics:
            annual_total = yearly_data[f'Annual Total {metric}'].iloc[0]
            multi_year_total = yearly_data[f'Multi-Year Total {metric}'].iloc[-1] if pd.notnull(yearly_data[f'Multi-Year Total {metric}'].iloc[-1]) else "NA"
            row_values = list(yearly_data[metric]) + [format_numbers(annual_total), format_numbers(multi_year_total)]
            table.loc[metric] = row_values

        # Apply conditional formatting for the Census row
        def highlight_census(row):
            if row.name == 'Census':
                return ['background-color: red' if v != "NA" and int(v) >= int(eligible_population_input) else '' for v in row]
            return ['' for _ in row]

        styled_table = table.style.apply(highlight_census, axis=1)

        html_output += f"<h3>Year {year}</h3>"
        html_output += styled_table.to_html()

    return html_output


# Generate projections
df = calculate_projections_fixed(eligible_population, target_enrollment_percent, initial_enrollment_percent, 
                                  monthly_growth_rate, monthly_attrition_rate, hospice_conversion_rate, projection_months)

# Calculate annual and multi-year totals
df = calculate_annual_and_multi_year_totals(df)

# Visualize projections
html_result_fixed = visualize_projections_fixed(df)
display(HTML(html_result_fixed))

,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,Month 9,Month 10,Month 11,Month 12,Annual Total,Multi-Year Total
New Enrollments,127,16,17,18,20,21,23,25,26,28,30,33,384,NA
Departures,0,6,7,7,8,8,9,10,11,11,12,13,102,NA
Census,127,137,147,158,170,183,197,212,227,244,262,282,NA,NA
Hospice Conversions,0,2,3,3,3,3,3,4,4,4,4,5,38,NA
,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,Month 9,Month 10,Month 11,Month 12,Annual Total,Multi-Year Total
New Enrollments,35,38,41,44,47,51,54,58,63,68,73,78,650,"7,800"
Departures,14,15,16,18,19,20,22,23,25,27,29,31,259,"3,108"
Census,303,326,351,377,405,436,468,503,541,582,626,673,NA,NA
Hospice Conversions,5,6,6,7,7,7,8,9,9,10,11,11,96,"1,152"
,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,Month 9,Month 10,Month 11,Month 12,Annual Total,Multi-Year Total


## Step 1: Clinical Staff Sourcing

Advanced Practice Provider (NP/PA)

In [11]:
salaried_input = '50'

In [12]:
hourly_contracted_input = '25'

In [13]:
borrowed_input = '25'

Chaplain

In [14]:
salaried_chaplain = '0'

In [15]:
hourly_chaplain = '0'

In [16]:
borrowed_chaplain = '100'

Collaborating/Supervisory Physician

In [17]:
salaried_physician = '100'

In [18]:
hourly_physician = '0'

In [19]:
borrowed_physician = '0'

Community Health Worker (Non-medical)

In [20]:
salaried_health_worker = '100'

In [21]:
hourly_health_worker = '0'

In [22]:
borrowed_health_worker = '0'

Healthcare Social Worker

In [23]:
salaried_social_worker = '100'

In [24]:
hourly_social_worker = '0'

In [25]:
borrowed_social_worker = '0'

Home Health Aide

In [26]:
salaried_hh_aide = '100'

In [27]:
hourly_hh_aide = '0'

In [28]:
borrowed_hh_aide = '0'

RN Case Manager

In [29]:
salaried_rn_case_manager = '100'

In [30]:
hourly_rn_case_manager = '0'

In [31]:
borrowed_rn_case_manager = '0'

PharmD

In [32]:
salaried_pharmad = '0'

In [33]:
hourly_pharmd = '100'

In [34]:
borrowed_pharmd = '0'

Other

In [35]:
salaried_other = '0'

In [36]:
hourly_other = '100'

In [37]:
borrowed_other = '0'

In [38]:
import pandas as pd

# Constants for default percentages
default_percentages = {
    "Advanced Practice Provider (NP/PA)": [50, 25, 25],
    "Chaplain": [0, 0, 100],
    "Collaborating/Supervisory Physician": [100, 0, 0],
    "Community Health Worker (Non-medical)": [100, 0, 0],
    "Healthcare Social Worker": [100, 0, 0],
    "Home Health Aide": [100, 0, 0],
    "RN Case Manager": [100, 0, 0],
    "PharmD": [0, 100, 0],
    "Psychologist": [0, 100, 0],
    "Other": [0, 100, 0]
}

# User inputs for each discipline
discipline_inputs = {
    "Advanced Practice Provider (NP/PA)": {"salaried_input": 50, "hourly_contracted_input": 25, "borrowed_input": 25},
    "Chaplain": {"salaried_chaplain": 0, "hourly_chaplain": 0, "borrowed_chaplain": 100},
    "Collaborating/Supervisory Physician": {"salaried_physician": 100, "hourly_physician": 0, "borrowed_physician": 0},
    "Community Health Worker (Non-medical)": {"salaried_health_worker": 100, "hourly_health_worker": 0, "borrowed_health_worker": 0},
    "Healthcare Social Worker": {"salaried_social_worker": 100, "hourly_social_worker": 0, "borrowed_social_worker": 0},
    "Home Health Aide": {"salaried_hh_aide": 100, "hourly_hh_aide": 0, "borrowed_hh_aide": 0},
    "RN Case Manager": {"salaried_rn_case_manager": 100, "hourly_rn_case_manager": 0, "borrowed_rn_case_manager": 0},
    "PharmD": {"salaried_pharmd": 0, "hourly_pharmd": 100, "borrowed_pharmd": 0},
    "Psychologist": {"salaried_input": 0, "hourly_contracted_input": 100, "borrowed_input": 0},
    "Other": {"salaried_other": 0, "hourly_other": 100, "borrowed_other": 0}
}

def collect_staff_sourcing():
    staff_sourcing = []
    print("\n### Step 1: Clinical Staff Sourcing ###\n")
    
    for discipline, inputs in discipline_inputs.items():
        # Reference default percentages
        default = default_percentages[discipline]
        
        # Fetch user inputs or fall back to defaults
        salaried = inputs.get("salaried_input", default[0]) / 100
        hourly_contracted = inputs.get("hourly_contracted_input", default[1]) / 100
        borrowed = inputs.get("borrowed_input", default[2]) / 100

        # Calculate total and adjust if needed
        total = salaried + hourly_contracted + borrowed
        if total != 1.0:
            print(f"Adjusting {discipline} percentages to sum to 100%. Current total: {total:.2f}")
            correction_factor = 1.0 / total
            salaried *= correction_factor
            hourly_contracted *= correction_factor
            borrowed *= correction_factor

        staff_sourcing.append({
            "Discipline": discipline,
            "Salaried (%)": round(salaried * 100, 2),
            "Hourly Contracted (%)": round(hourly_contracted * 100, 2),
            "Borrowed (%)": round(borrowed * 100, 2),
            "Total (%)": round(total * 100, 2)
        })
    
    return pd.DataFrame(staff_sourcing)

# Collect and display the data
staff_sourcing = collect_staff_sourcing()

# Convert the DataFrame to HTML
html_table = staff_sourcing.to_html(classes='table table-striped table-bordered table-hover', index=False)

# Display the HTML table
from IPython.display import display, HTML

display(HTML(html_table))


### Step 1: Clinical Staff Sourcing ###



Discipline,Salaried (%),Hourly Contracted (%),Borrowed (%),Total (%)
Advanced Practice Provider (NP/PA),50.0,25.0,25.0,100.0
Chaplain,0.0,0.0,100.0,100.0
Collaborating/Supervisory Physician,100.0,0.0,0.0,100.0
Community Health Worker (Non-medical),100.0,0.0,0.0,100.0
Healthcare Social Worker,100.0,0.0,0.0,100.0
Home Health Aide,100.0,0.0,0.0,100.0
RN Case Manager,100.0,0.0,0.0,100.0
PharmD,0.0,100.0,0.0,100.0
Psychologist,0.0,100.0,0.0,100.0
Other,0.0,100.0,0.0,100.0


### Step 2: Salaries and Fringe for Employed Clinical Staff

In [39]:
fringe_rate_input = '0.32'

In [40]:
work_hours_per_year_input = '2080'

In [41]:
# Step 2: Salaries and Fringe for Employed Clinical Staff
def calculate_salaries_and_fringes(staff_sourcing):
    salary_data = []
    for _, row in staff_sourcing.iterrows():
        discipline = row["Discipline"]

        # Inputs for calculations
        annual_salary = Annual_Salary_per_FTE_input.get(discipline, 0)  # Default to 0 if not provided
        fringe_rate = fringe_rate_input
        work_hours = work_hours_per_year_input

        # Business rules for calculations
        fringe = round(annual_salary * fringe_rate, 2)
        loaded_cost = round(annual_salary + fringe, 2)
        hourly_rate = round(loaded_cost / work_hours, 2) if annual_salary > 0 else 0

        # Append calculated values to the result
        salary_data.append({
            "Discipline": discipline,
            "Annual Salary (Maine Mean) per FTE": annual_salary,
            "Fringe per FTE": fringe,
            "Loaded Cost per FTE": loaded_cost,
            "Computed Hourly Rate": hourly_rate
        })
    
    # Convert results to a pandas DataFrame
    return pd.DataFrame(salary_data)

# Inputs from the business rules
fringe_rate_input = 0.32  # 32%
work_hours_per_year_input = 2080  # Work hours in a work year

# Salary inputs (Maine Mean per FTE)
Annual_Salary_per_FTE_input = {
    "Advanced Practice Provider (NP/PA)": 127865,
    "Chaplain": 61150,
    "Collaborating/Supervisory Physician": 229981,
    "Community Health Worker (Non-medical)": 48810,
    "Healthcare Social Worker": 66130,
    "Home Health Aide": 36750,
    "RN Case Manager": 84340,
    "PharmD": 133540,
    "Psychologist": 117710,
    "Other:": 0  # Default value for "Other"
}

# Execute Step 2
salary_data = calculate_salaries_and_fringes(staff_sourcing)

# Display results as an HTML table
from IPython.display import display, HTML
display(HTML(salary_data.to_html(classes='table table-striped table-bordered table-hover', index=False)))

Discipline,Annual Salary (Maine Mean) per FTE,Fringe per FTE,Loaded Cost per FTE,Computed Hourly Rate
Advanced Practice Provider (NP/PA),127865,40916.80,168781.80,81.15
Chaplain,61150,19568.00,80718.00,38.81
Collaborating/Supervisory Physician,229981,73593.92,303574.92,145.95
Community Health Worker (Non-medical),48810,15619.20,64429.20,30.98
Healthcare Social Worker,66130,21161.60,87291.60,41.97
Home Health Aide,36750,11760.00,48510.00,23.32
RN Case Manager,84340,26988.80,111328.80,53.52
PharmD,133540,42732.80,176272.80,84.75
Psychologist,117710,37667.20,155377.20,74.70
Other,0,0.00,0.00,0.00


### Step 3: Hourly Rate for Contracted Clinical Staff

In [42]:
# Step 3: Calculate Hourly Rate for Contracted Clinical Staff
def calculate_hourly_rates(salary_data):
    contracted_rates = []
    
    # Get computed hourly rate for each discipline
    for _, row in salary_data.iterrows():
        discipline = row["Discipline"]
        computed_hourly_rate = row["Computed Hourly Rate"]
        
        # For "Other:", set hourly rate to 0
        if discipline == "Other:":
            hourly_rate = 0
        else:
            hourly_rate = computed_hourly_rate
        
        contracted_rates.append({
            "Discipline": discipline,
            "Hourly Rate": round(hourly_rate, 2)
        })
    
    # Convert results to a pandas DataFrame
    return pd.DataFrame(contracted_rates)

# Execute Step 3
contracted_rates = calculate_hourly_rates(salary_data)

# Display results as an HTML table
display(HTML(contracted_rates.to_html(classes='table table-striped table-bordered table-hover', index=False)))


Discipline,Hourly Rate
Advanced Practice Provider (NP/PA),81.15
Chaplain,38.81
Collaborating/Supervisory Physician,145.95
Community Health Worker (Non-medical),30.98
Healthcare Social Worker,41.97
Home Health Aide,23.32
RN Case Manager,53.52
PharmD,84.75
Psychologist,74.70
Other,0.00


### Admin & Overhead Costs

### Step 1:  Select flat rate or detailed approach for administrative and overhead costs

In [43]:
pip install ipywidgets


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [57]:
flat_rate_input = 'Yes'

Admin/Support

In [45]:
census_per_fte_adm_sup = '  500 '

In [46]:
annual_salary_without_fringe_per_fte_adm_sup = ' 43780 '

Biller

In [47]:
census_per_fte_biller = '  1500 '

In [48]:
annual_salary_without_fringe_per_fte_biller = ' 46585 '

Senior Leadership

In [49]:
census_per_fte_senior = '3000'

In [50]:
annual_salary_without_fringe_per_fte_senior = '156260 '

Practice Management

In [51]:
census_per_fte_practice = '1000'

In [52]:
annual_salary_without_fringe_per_fte_practice = ' 118410 '

Other

In [53]:
census_per_fte_other = '0'

In [54]:
annual_salary_without_fringe_per_fte_other = '0'

In [60]:
import pandas as pd
from IPython.display import HTML, display

# inputs
flat_rate = 20
census_per_fte_adm_sup = int(census_per_fte_adm_sup)
annual_salary_without_fringe_per_fte_adm_sup = int(annual_salary_without_fringe_per_fte_adm_sup)
census_per_fte_biller = int(census_per_fte_biller)
annual_salary_without_fringe_per_fte_biller = int(annual_salary_without_fringe_per_fte_biller)
census_per_fte_senior = int(census_per_fte_senior)
annual_salary_without_fringe_per_fte_senior = int(annual_salary_without_fringe_per_fte_senior)
census_per_fte_other = int(census_per_fte_other)
annual_salary_without_fringe_per_fte_other = int(annual_salary_without_fringe_per_fte_other)
fringe_rate_input = float(fringe_rate_input)  

# Function to display dataframe as HTML
def display_dataframe_as_html(df):
    """Display the DataFrame as an HTML table with styling."""
    display(HTML(df.to_html(classes='table table-striped table-bordered table-hover', index=False)))

# Step 1: Flat rate table display
def display_step_1_table(flat_rate, clinical_costs_total, admin_costs):
    """
    Display the styled table for Step 1: Flat Rate or Detailed Approach.
    """
    data = {
        "Input/Output": ["Input", "Input", "Conditional Input", "Output"],
        "Description": ["Use flat rate?", "Flat rate percentage (%)", "Clinical costs (USD)", "Administrative costs (USD)"],
        "Value": ["Yes", f"{flat_rate}%", f"${clinical_costs_total:,.2f}", f"${admin_costs:,.2f}"]
    }
    df = pd.DataFrame(data)
    display_dataframe_as_html(df)  # Display the styled table as HTML

# Step 2A: Administrative personnel detailed approach
def display_admin_costs_table():
    """
    Display the detailed costs table for administrative personnel.
    """
    # Calculate fringe for each role
    fringe_per_fte_adm_sup = annual_salary_without_fringe_per_fte_adm_sup * fringe_rate_input
    fringe_per_fte_biller = annual_salary_without_fringe_per_fte_biller * fringe_rate_input
    fringe_per_fte_senior = annual_salary_without_fringe_per_fte_senior * fringe_rate_input
    fringe_per_fte_other = annual_salary_without_fringe_per_fte_other * fringe_rate_input
    
    roles_data = [
        ("Admin/Support", census_per_fte_adm_sup, annual_salary_without_fringe_per_fte_adm_sup, fringe_per_fte_adm_sup),
        ("Biller", census_per_fte_biller, annual_salary_without_fringe_per_fte_biller, fringe_per_fte_biller),
        ("Senior Leadership", census_per_fte_senior, annual_salary_without_fringe_per_fte_senior, fringe_per_fte_senior),
        ("Practice Management", census_per_fte_senior, annual_salary_without_fringe_per_fte_senior, fringe_per_fte_senior),
        ("Other", census_per_fte_other, annual_salary_without_fringe_per_fte_other, fringe_per_fte_other)
    ]
    
    data = []
    for role, census_per_fte, salary, fringe_per_fte in roles_data:
        fully_loaded_per_fte = salary + fringe_per_fte  # Fully loaded cost = salary + fringe
        
        # Append to data for DataFrame
        data.append([role, census_per_fte, f"${salary:,.2f}", f"${fringe_per_fte:,.2f}", f"${fully_loaded_per_fte:,.2f}"])
    
    df = pd.DataFrame(data, columns=["Role", "Census per FTE", "Annual salary without fringe per FTE", 
                                     "Fringe (rate from prior tab) per FTE", "Fully loaded per FTE"])
    display_dataframe_as_html(df)  # Display the styled table as HTML

# User input for flat rate (yes/no) and other values
flat_rate_input = str(flat_rate_input) 

# Default values for clinical costs and flat rate
flat_rate = float(flat_rate)  # Convert flat_rate to float for percentage calculations
clinical_costs_total = 10000.0  # Total clinical costs (USD)
admin_costs = clinical_costs_total * (flat_rate / 100)  # Administrative costs based on flat rate

# Step 1 Logic: Flat rate vs Detailed approach
if flat_rate_input.lower() == "yes":
    print("Displaying flat rate table:")
    display_step_1_table(flat_rate, clinical_costs_total, admin_costs)
else:
    print("Displaying detailed administrative costs table:")
    display_admin_costs_table()


Displaying detailed administrative costs table:


Role,Census per FTE,Annual salary without fringe per FTE,Fringe (rate from prior tab) per FTE,Fully loaded per FTE
Admin/Support,500,"$43,780.00","$14,009.60","$57,789.60"
Biller,1500,"$46,585.00","$14,907.20","$61,492.20"
Senior Leadership,3000,"$156,260.00","$50,003.20","$206,263.20"
Practice Management,3000,"$156,260.00","$50,003.20","$206,263.20"
Other,0,$0.00,$0.00,$0.00


### Scenario 1-Adult (2)

In [54]:
clinical_hour_pmpm = '0.30'

In [55]:
non_clinical_hours_pmpm = '0.30'

In [61]:
import pandas as pd
from IPython.display import display, HTML

# Constants for default percentages
default_percentages = {
    "Advanced Practice Provider (NP/PA)": [0.30, 0.30],
    "Chaplain": [0.15, 0.15],
    "Collaborating/Supervisory Physician": [0.15, 0.15],
    "Community Health Worker (Non-medical)": [0.75, 0.25],
    "Healthcare Social Worker": [0.65, 0.50],
    "Home Health Aide": [0.25, 0.25],
    "RN Case Manager": [2.00, 1.00],
    "PharmD": [0.10, 0.05],
    "Psychologist or Mental Health Practitioner": [0.25, 0.15],
    "Other": [0.00, 0.00],
}

# User inputs for each discipline
discipline_inputs = {
    "Advanced Practice Provider (NP/PA)": {"clinical_hour_pmpm": 0.30, "non_clinical_hours_pmpm": 0.30},
    "Chaplain": {"clinical_hour_pmpm": 0.15, "non_clinical_hours_pmpm": 0.15},
    "Collaborating/Supervisory Physician": {"clinical_hour_pmpm": 0.15, "non_clinical_hours_pmpm": 0.15},
    "Community Health Worker (Non-medical)": {"clinical_hour_pmpm": 0.75, "non_clinical_hours_pmpm": 0.25},
    "Healthcare Social Worker": {"clinical_hour_pmpm": 0.65, "non_clinical_hours_pmpm": 0.50},
    "Home Health Aide": {"clinical_hour_pmpm": 0.25, "non_clinical_hours_pmpm": 0.25},
    "RN Case Manager": {"clinical_hour_pmpm": 2.00, "non_clinical_hours_pmpm": 1.00},
    "PharmD": {"clinical_hour_pmpm": 0.10, "non_clinical_hours_pmpm": 0.05},
    "Psychologist or Mental Health Practitioner": {"clinical_hour_pmpm": 0.25, "non_clinical_hours_pmpm": 0.15},
    "Other": {"clinical_hour_pmpm": 0.00, "non_clinical_hours_pmpm": 0.00},
}

def get_default_or_user_input(discipline, key):
    """Retrieve user input if available, otherwise use default values."""
    user_value = discipline_inputs.get(discipline, {}).get(key)
    if user_value is not None:
        return user_value
    # Default values fall back to default_percentages
    return default_percentages[discipline][0 if key == "clinical_hour_pmpm" else 1]

# Function to create the dataframe with calculated values
def create_dataframe(staff_sourcing):
    """Create a DataFrame with default or user-provided inputs and calculate totals."""
    data = []
    
    for idx, (discipline, percentages) in enumerate(default_percentages.items()):
        # Get user input or default values for clinical and non-clinical hours
        clinical_hours = get_default_or_user_input(discipline, "clinical_hour_pmpm")
        non_clinical_hours = get_default_or_user_input(discipline, "non_clinical_hours_pmpm")
        
        # Calculate total hours PMPM
        total_hours = clinical_hours + non_clinical_hours
        
        # Get staffing data for the discipline from staff_sourcing
        salaried_pct = staff_sourcing["Salaried (%)"][idx] / 100
        hourly_contracted_pct = staff_sourcing["Hourly Contracted (%)"][idx] / 100
        
        # Retrieve computed hourly rate (assuming already calculated for each discipline)
        computed_hourly_rate = contracted_rates["Hourly Rate"][idx]  # Get the rate from contracted_rates
        
        # Calculate PMPM cost for clinical staff based on staffing percentages and computed hourly rates
        pmpm_cost = total_hours * (
            (salaried_pct * computed_hourly_rate) +
            (hourly_contracted_pct * computed_hourly_rate)
        )
        
        # Append calculated values
        data.append({
            "Discipline": discipline,
            "Clinical hours PMPM": clinical_hours,
            "Non-clinical hours PMPM": non_clinical_hours,
            "Total hours PMPM": total_hours,
            "PMPM cost for clinical staff": pmpm_cost
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Add total row
    total_row = pd.DataFrame([{
        "Discipline": "Total",
        "Clinical hours PMPM": df["Clinical hours PMPM"].sum(),
        "Non-clinical hours PMPM": df["Non-clinical hours PMPM"].sum(),
        "Total hours PMPM": df["Total hours PMPM"].sum(),
        "PMPM cost for clinical staff": df["PMPM cost for clinical staff"].sum()
    }])
    
    return pd.concat([df, total_row], ignore_index=True)

# Function to display dataframe as HTML
def display_dataframe_as_html(df):
    """Display the DataFrame as an HTML table with styling."""
    display(HTML(df.to_html(classes='table table-striped table-bordered table-hover', index=False)))

# Staff_sourcing and contracted_rates are already calculated
# Create the dataframe with the sourced data
final_df = create_dataframe(staff_sourcing)

# Display the results as an HTML table
display_dataframe_as_html(final_df)


Discipline,Clinical hours PMPM,Non-clinical hours PMPM,Total hours PMPM,PMPM cost for clinical staff
Advanced Practice Provider (NP/PA),0.30,0.30,0.60,36.5175
Chaplain,0.15,0.15,0.30,0.0000
Collaborating/Supervisory Physician,0.15,0.15,0.30,43.7850
Community Health Worker (Non-medical),0.75,0.25,1.00,30.9800
Healthcare Social Worker,0.65,0.50,1.15,48.2655
Home Health Aide,0.25,0.25,0.50,11.6600
RN Case Manager,2.00,1.00,3.00,160.5600
PharmD,0.10,0.05,0.15,12.7125
Psychologist or Mental Health Practitioner,0.25,0.15,0.40,29.8800
Other,0.00,0.00,0.00,0.0000


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=da9886f7-9ffa-479d-8dd2-6f3d216a5138' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>